# Geometric square
https://geodata-frisco.hub.arcgis.com/datasets/frisco::citylimits/explore

## Frisco

In [ ]:
import pandas as pd
from pyproj import Proj, transform

data = pd.read_csv('Frisco_1km.csv')
inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
def f(x):
    return transform(inProj, outProj, x['X'], x['Y'])

coordinates = data.apply(lambda x: f(x), result_type='expand', axis=1)
data = pd.concat([data, coordinates], axis=1)

data.to_csv('./convertedlocs_Frisco.csv',index=None)

c:\Users\jacob\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\jacob\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\jacob\AppData\Local\Temp/ipykernel_20924/792836779.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/go

In [2]:
import requests
import json
import time
import pickle
import pandas as pd

+ Replace API_KEY
+ This endpoint is for business search

In [34]:
pos1=data.iloc[0][0]
pos2=data.iloc[0][1]

In [1]:
API_KEY = 'l6CQDw10V0hgHFzX_u14o4ljYf8ok6hEp6o3uJQRfPB805dh24sAKipHyPCE-8HsMoYeRuLbiXnNnreoS_1f101cLDBLmMnHzbFGict5L036QHKw1dzqfMYGCTtXY3Yx'
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

### Junk TESTS

In [48]:
PARAMETERS = {'term': 'restaurant',
                  'offset': 0, # this offset is to prevent duplicates
                  'limit': 50, 
                  'latitude':float(pos2),
                  'longitude':float(pos1),
                  'radius':1000}

In [49]:
response = requests.get(url = ENDPOINT, params= PARAMETERS, headers = HEADERS)

200

## Functions

In [6]:
def build_PARAMS(lat,long,radius,offset):
    params = {'term': 'restaurant',
                  'offset': offset, # this offset is to prevent duplicates
                  'limit': 50, 
                  'latitude':float(lat),
                  'longitude':float(long),
                  'radius':radius}
    return(params)

def handle_good_return(response,output):
    business_data = response.json()
    output['businesses'] += business_data['businesses']
    return(output)

def handle_bad_response(lat,long,offset_val,output):
    retry_flag=True
    for i in range(0,3):
        if retry_flag:
            time.sleep(10)
            PARAMETERS=build_PARAMS(lat,long,1000,offset_val)
            retry_request=requests.get(url = ENDPOINT, params= PARAMETERS, 
                headers = HEADERS)
            code=retry_request.status_code
            if code==200:
                retry_flag=False
                output=handle_good_return(retry_request,output)
                break
            else:
                print(f'Attempt number {i} failed will retry')
                remaining=int(retry_request.headers['ratelimit-remaining'])
                if remaining <= 10:
                    print(f'Maybe hitting daily limit will sleep until then and retry')
                    handle_sleep(retry_request)
                pass
    if retry_flag:
        raise Exception (f'Something is broken here is lat\
            {lat}, long {long}, offset_val {offset_val}',)
    else:
        return(output)

def handle_sleep(retry_request):
    reset_time=pd.to_datetime(retry_request.headers['ratelimit-resettime'])
    now=pd.Timestamp.utcnow()
    remaining_time=reset_time-now
    print(f'Should reset in {remaining_time}')
    sleeptime=remaining_time.seconds
    time.sleep(sleeptime)


def handle_multiple_queries(lat,long,li_of_offsets,output,snagflag):
    for offset_val in li_of_offsets:
        PARAMETERS=build_PARAMS(lat,long,1000,offset_val)
        return_request=requests.get(url = ENDPOINT, params= PARAMETERS, 
        headers = HEADERS)
        code=return_request.status_code
        if code==200:
            output=handle_good_return(return_request,output)
        else:
            print(f'Request Failed, {code}, going to retry in 10 seconds')
            try:
                output=handle_bad_response(lat,long,offset_val,output,snagflag)
            except:
                snagflag=True
                print('Hit a Snag')
                break
    return(output,snagflag)





In [8]:
output = {}
output['businesses'] = []
last_remains=5000
try_num=0
snagflag=False
# Okay we're going to iterate over the geographic points
for _,row in data.iterrows():
    #backwards so 0 last
    lat=row[1]
    long=row[0]
    offset=0
    print(f'Starting query on ({lat},{long})')
    print(f'Output Size is {len(output["businesses"])}')
    PARAMETERS=build_PARAMS(lat,long,50_000,offset)
    starting_response = requests.get(url = ENDPOINT, params= PARAMETERS, headers = HEADERS)
    #Congrats it worked
    if starting_response.status_code==200:
        #write it out before doing anything else
        output=handle_good_return(starting_response,output)
        #Now what happens if we have more than 50 results
        length=starting_response.json()['total']
        #1000 is the limit so if its over we need to handle it differently
        # Easy Case
        if length>50 and length<=1000:
            # build out offset list
            li_of_offsets=list(range(50,length,50))
            #build output in seperate func
            output,snagflag=handle_multiple_queries(lat,long,li_of_offsets,output,snagflag)
            if snagflag:
                raise Exception
        elif length>1000:
            #need to reset radius maybe?
            raise Exception
    else:
        output=handle_bad_response(lat,long,offset,output)
    

    

Starting query on (33.219369763091024,-96.92274796186854)
Output Size is 0
Starting query on (33.219369763091024,-96.91376480902734)
Output Size is 11
Starting query on (33.219369763091024,-96.90478165618616)
Output Size is 18
Starting query on (33.219369763091024,-96.89579850334496)
Output Size is 26
Starting query on (33.219369763091024,-96.88681535050375)
Output Size is 31
Starting query on (33.219369763091024,-96.87783219766256)
Output Size is 43
Starting query on (33.219369763091024,-96.86884904482136)
Output Size is 60
Starting query on (33.219369763091024,-96.85986589198018)
Output Size is 70
Starting query on (33.219369763091024,-96.85088273913898)
Output Size is 70
Starting query on (33.219369763091024,-96.84189958629779)
Output Size is 70
Starting query on (33.219369763091024,-96.83291643345659)
Output Size is 71
Starting query on (33.219369763091024,-96.82393328061539)
Output Size is 73
Starting query on (33.219369763091024,-96.8149501277742)
Output Size is 75
Starting query

In [9]:
pd.DataFrame(output['businesses']).to_csv('YelpData_Frisco_v2.csv')

In [27]:
import ast
df_mine=pd.DataFrame(output['businesses'])
df_mine=df_mine[df_mine['id'].duplicated(keep='first')]
df_an=pd.read_csv('./business_search_Frisco_1000only.csv')
missing=df_an[~(df_an['id'].isin(df_mine['id']))]
sample_missing=missing.sample(n=10)
for i in sample_missing['coordinates']:
    i=ast.literal_eval(i)
    print(i['latitude'],',',i['longitude'])

33.0407651 , -96.71458
33.17373 , -96.71615
33.003164 , -96.976663
33.035858 , -96.897517
32.98521 , -96.99273
33.027031035904926 , -96.8313105042077
33.1558868404851 , -96.9439333147799
33.06828 , -96.88969
32.96301666960503 , -96.78989436123085
32.999487 , -96.825642


Most of these are showing out the city limits its no wonder

+ Write output to text file

In [4]:
# how many businesses?
count = 0
for x in output:
    if isinstance(output[x], list):
        count += len(output[x])
print(count)

1000


Get list of business ids to feed the endpoint for business details. One call = 1 business ID

In [11]:
my_list = output['businesses']
business_id = [d['id'] for d in my_list]

business_details = []

for i in business_id:
    b_id = i
    ENDPOINT = 'https://api.yelp.com/v3/businesses/%s' % b_id
    response = requests.get(url = ENDPOINT, headers = HEADERS)
    business_data = response.json()
    business_details.append(business_data)

Write file to txt

In [15]:
with open('business_details_enterprise.txt', 'w',encoding="utf-8") as fp:
    for item in business_details:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done
